# Mirascope Quickstart Guide

This guide provides a comprehensive introduction to the key features of the Mirascope library, designed for developers who are new to working with Large Language Models (LLMs). Each section includes detailed explanations, code examples, and practical advice.

## Table of Contents
1. [Setup](#Setup)
2. [Prompt Templates (prompt)](#Prompt-Templates)
3. [Basic LLM Call (call)](#Basic-LLM-Call)
4. [Tools](#Tools)
5. [Asynchronous Processing (Async)](#Asynchronous-Processing)
6. [Streaming Responses (streams)](#Streaming-Responses)
7. [Response Models (response_model)](#Response-Models)
8. [Dynamic Configuration & Chaining](#Dynamic-Configuration-and-Chaining)
9. [JSON Mode](#JSON-Mode)
10. [Output Parsers](#Output-Parsers)

Let's dive into each of these features!

## Setup

Mirascope supports integration with various LLM providers including OpenAI, Anthropic, and Gemini. Let's start by installing Mirascope and its dependencies:

In [ ]:
!pip install "mirascope[openai]"

# Additional installation for other providers
# !pip install "mirascope[anthropic, gemini]"

This command installs Mirascope with OpenAI integration. For other providers, you may need to install additional extras (e.g., `pip install mirascope[anthropic]`).

Next, let's import the necessary modules and set up the API keys for the providers we'll be using:

In [ ]:
import os

from mirascope.core import openai, prompt_template

# Import additional modules for other providers
# from mirascope.core import anthropic, gemini

In [ ]:
# Set the appropriate API key for the provider you're using
os.environ["OPENAI_API_KEY"] = "your-openai-api-key-here"
os.environ["ANTHROPIC_API_KEY"] = "your-anthropic-api-key-here"
os.environ["GOOGLE_API_KEY"] = "your-google-api-key-here"

Setting API keys as environment variables is recommended for better security and portability across different environments.

## Prompt Templates

The `prompt_template` decorator in Mirascope is a powerful tool for creating dynamic and reusable prompts. It makes prompt engineering more flexible and manageable.

Here's a basic example of how to define a prompt template:

In [ ]:
@prompt_template("What is the capital of {country}?")
def get_capital_prompt(country: str):
    pass


# This function doesn't actually call an LLM yet, it just defines the prompt
print(get_capital_prompt("Japan"))

In this example:
1. The `@prompt_template` decorator defines the structure of the prompt.
2. `{country}` is a placeholder that gets replaced with the argument passed to the function.
3. The actual function body (`pass`) is ignored; only the prompt structure is defined.

You can use more complex prompts with multiple variables:

In [ ]:
@prompt_template("""
SYSTEM: You are a helpful assistant that provides information about {topic}.
USER: What are the key points to understand about {subtopic} in {topic}?
""")
def get_info_prompt(topic: str, subtopic: str):
    pass


print(get_info_prompt("artificial intelligence", "machine learning"))

This advanced example:
1. Includes "SYSTEM" and "USER" roles in the prompt, providing different contexts.
2. Uses multiple variables (`{topic}` and `{subtopic}`) to create a more specific and dynamic prompt.

Prompt templates improve code readability, maintain consistency in prompts, and make it easier to experiment with and modify prompts for effective prompt engineering.

## Basic LLM Call

Now that we understand prompt templates, let's see how to use them with the `call` decorator to actually interact with an LLM. The `call` decorator in Mirascope transforms regular Python functions with prompt templates into LLM API calls.

Here's a basic example combining a prompt template with an LLM call:

In [ ]:
@openai.call("gpt-4o-mini")
@prompt_template("What is the capital of {country}?")
def get_capital(country: str):
    pass


response = get_capital("Japan")
print(response.content)

# Additional response information
print(f"Model used: {response.model}")
print(f"Token usage: {response.usage}")
print(f"Finish reason: {response.finish_reasons}")

In this example:
1. The `@prompt_template` decorator defines the structure of the prompt.
2. The `@openai.call("gpt-4o-mini")` decorator transforms the function into an LLM call using OpenAI's GPT-4 model.
3. When the function is called, it combines the prompt template with the provided arguments and sends the resulting prompt to the LLM.
4. The function returns a response object containing the LLM's output and additional metadata.

This combination of prompt templates and the call decorator makes it easy to create flexible and reusable LLM interactions.

## Tools

Tools in Mirascope allow you to extend the capabilities of LLMs by providing access to custom functionality. This feature significantly enhances the LLM's ability to perform complex tasks or generate specific outputs.

Here's an example of how to define and use a custom tool, continuing with our capital city theme:

In [ ]:
from mirascope.core import BaseTool, openai, prompt_template


class FormatCapital(BaseTool):
    city: str
    country: str

    def call(self) -> str:
        return f"The capital of {self.country} is {self.city}."


@openai.call("gpt-4o-mini", tools=[FormatCapital])
@prompt_template("What is the capital of {country}?")
def get_capital(country: str): ...


response = get_capital("France")
if tools := response.tools:
    for tool in tools:
        print(tool.call())
else:
    print(response.content)

Let's break down this example:

1. We define a custom tool `FormatCapital` that inherits from `BaseTool`. This tool takes a city and country, and formats them into a sentence stating the capital.

2. The `FormatCapital` class has two attributes (`city` and `country`) and a `call` method that returns the formatted string.

3. In the `get_capital` function, we use the `@openai.call` decorator and include our `FormatCapital` tool in the `tools` parameter.

4. The LLM can now use this tool when generating its response. If it decides to use the tool, it will provide values for `city` and `country`.

5. When we call `get_capital`, we check if any tools were used in the response. If so, we call each tool and print its output. Otherwise, we print the direct content of the response.

This approach allows the LLM to generate structured data (in this case, a capital city and its country) in a format that's easy for your application to process.

Tools can be used for various purposes, such as:
- Formatting output in a specific way
- Performing calculations (e.g., population density given area and population)
- Accessing external data sources (e.g., fetching up-to-date information about a city)
- Implementing custom logic that the LLM can leverage (e.g., determining the continent based on the country)

By providing tools, you give the LLM additional capabilities that it can use to generate more accurate, formatted, or context-aware responses. In this case, the `FormatCapital` tool ensures that the capital information is always presented in a consistent format.

## Asynchronous Processing

Mirascope supports asynchronous processing, allowing for efficient parallel execution of multiple LLM calls. This is particularly useful when handling numerous queries or when you need to minimize response times.

Here's a basic example of asynchronous calls:

In [ ]:
import asyncio


@openai.call("gpt-4o-mini")
@prompt_template("What is the capital of {country}?")
async def get_capital_async(country: str):
    pass


async def main():
    countries = ["France", "Japan", "Brazil"]
    tasks = [get_capital_async(country) for country in countries]
    results = await asyncio.gather(*tasks)
    for country, result in zip(countries, results, strict=False):
        print(f"The capital of {country} is {result.content}")


# Use asyncio.run() to execute in the main thread
# asyncio.run(main())

# Use the following code if running in a Jupyter notebook
await main()

This code:
1. Defines an asynchronous function using `async def`.
2. Uses `asyncio.gather()` to run multiple asynchronous tasks in parallel.
3. Processes the results in order, but the API calls themselves are made concurrently, reducing overall execution time.

## Streaming Responses

Streaming allows you to process LLM responses in real-time. This is particularly useful for generating longer responses or providing real-time feedback to users. Let's see how we can use streaming with our capital city example:

In [ ]:
@openai.call("gpt-4o-mini", stream=True)
@prompt_template(
    "Provide detailed information about the capital of {country}. Include facts about its history, culture, and significant landmarks."
)
def stream_capital_info(country: str):
    pass


for chunk, _ in stream_capital_info("France"):
    print(chunk.content, end="", flush=True)

In this example:

1. We enable streaming mode with the `stream=True` parameter in the `@openai.call` decorator.
2. Our prompt asks for detailed information about a capital city, which is likely to generate a longer response.
3. The response is returned in small chunks that can be processed immediately.
4. We use `end=""` and `flush=True` to display the output immediately without line breaks, creating a smooth streaming effect.

This approach has several advantages:

1. **Responsiveness**: Users see the beginning of the response immediately, rather than waiting for the entire response to be generated.
2. **Handling Long Outputs**: For detailed information like this, streaming allows you to start processing or displaying the response before it's fully generated.
3. **Real-time Interaction**: You could potentially process the chunks as they come in, allowing for real-time analysis or formatting of the response.

For example, you could modify the processing to format the output in real-time:

In [ ]:
import re


def format_capital_info(chunk: str) -> str:
    # Convert "History:" (and similar headers) to bold
    chunk = re.sub(r"(\w+):", r"**\1:**", chunk)
    # Add bullet points to lists
    chunk = re.sub(r"^(\s*)(\d+\.)", r"\1- ", chunk, flags=re.MULTILINE)
    return chunk


for chunk, _ in stream_capital_info("Japan"):
    formatted_chunk = format_capital_info(chunk.content)
    print(formatted_chunk, end="", flush=True)

This modification formats the streamed content in real-time, bold-ing headers and converting numbered lists to bullet points. This demonstrates how streaming can be combined with real-time processing to enhance the presentation of information as it's generated.

Streaming is particularly useful when dealing with longer, more detailed responses, such as comprehensive information about capital cities. It allows for a more interactive and responsive user experience, especially in applications where immediate feedback is valuable.

## Response Models

Response models allow you to structure and validate the output from LLMs. This feature enhances type safety and makes data manipulation easier.

Here's an example:

In [ ]:
from pydantic import BaseModel, field_validator


class Capital(BaseModel):
    city: str
    country: str
    population: int

    @field_validator("population")
    @classmethod
    def population_must_be_positive(cls, v):
        if v <= 0:
            raise ValueError("Population must be positive")
        return v


@openai.call("gpt-4o-mini", response_model=Capital)
@prompt_template("Provide information about the capital of {country}")
def get_capital_info(country: str):
    pass


try:
    result = get_capital_info("Japan")
    print(f"The capital of {result.country} is {result.city}")
    print(f"Population: {result.population}")
except ValueError as e:
    print(f"Validation error: {e}")

In this example:
1. We define a `Capital` model using Pydantic, specifying the structure and types of the expected data.
2. We use a field validator to ensure the population is always positive.
3. The `response_model` parameter in the `@openai.call` decorator tells Mirascope to parse the LLM's output into this structure.
4. We can then access the structured data directly, with type checking and validation already performed.

Response models are particularly useful when you need consistent, structured output from your LLM calls, especially for data-driven applications.

## Dynamic Configuration & Chaining

Dynamic configuration allows you to modify LLM call settings at runtime, while chaining enables you to create sequences of LLM calls.

Here's an example of dynamic configuration:

In [ ]:
@openai.call("gpt-4o-mini")
@prompt_template("Translate the following to {language}: {text}")
def translate(text: str, language: str):
    return {"temperature": 0.7 if language == "French" else 0.5}


response = translate("Hello, world!", "French")
print(response.content)

In this example, we're dynamically setting the `temperature` parameter based on the target language.

Now, let's look at an example of chaining:

In [ ]:
@openai.call("gpt-4o-mini")
@prompt_template("Summarize the following text: {text}")
def summarize(text: str):
    pass


@openai.call("gpt-4o-mini")
@prompt_template("Translate the following to French: {text}")
def translate_to_french(text: str):
    pass


original_text = "Recently, it's been really hot, so I’m thinking of gathering with friends to enjoy some champagne outdoors. I’m looking for something refreshing and light, especially a champagne that’s enjoyable even on a hot day. If possible, I’d love to know about any recommended champagne that’s perfect for relaxing with everyone."
summary = summarize(original_text)
french_summary = translate_to_french(summary.content)
print(french_summary.content)

This chain first summarizes a text, then translates the summary to French, demonstrating how you can combine multiple LLM calls to perform more complex tasks.

## JSON Mode

JSON mode allows you to directly parse LLM outputs as JSON. This is useful when you need structured data output.

Here's an example:

In [ ]:
@openai.call("gpt-4o-mini", json_mode=True)
@prompt_template("Provide information about {city} in JSON format")
def city_info(city: str):
    pass


response = city_info("Tokyo")
print(response.content)  # This will be a JSON-formatted string

By setting `json_mode=True`, we instruct the LLM to format its response as JSON, which can then be easily parsed and used in your application.

## Output Parsers

Output parsers allow you to process LLM responses in custom formats. This is useful when you need to extract specific information or transform the output in a particular way.

Here's an example:

In [ ]:
class CountryInfo(BaseModel):
    country: str
    capital: str


def parse_country_and_capital(response: openai.OpenAICallResponse) -> CountryInfo:
    country, capital = response.content.split(" -> ")
    return CountryInfo(country=country, capital=capital)


@openai.call("gpt-4o-mini", output_parser=parse_country_and_capital)
@prompt_template("What is the capital of {country}? the format 'country -> capital'.")
def country_and_capital(country: str): ...


country_info = country_and_capital("Australia")
print(f"{country_info=}")
print(f"The capital of {country_info.country} is {country_info.capital}")

In this example, we define a custom parser that splits the LLM's output into a list. The `output_parser` parameter in the `@openai.call` decorator applies this parser to the LLM's response.

This concludes our Quickstart Guide to Mirascope. We've covered the main features of the library, including basic calls, prompt templates, tools, asynchronous processing, streaming, response models, dynamic configuration, chaining, JSON mode, and output parsers. Each of these features can be combined and customized to create powerful, flexible AI applications. As you continue to work with Mirascope, you'll discover even more ways to leverage these capabilities in your projects.